In [40]:
# import preprocess as prep
import SLR_model
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess
# from preprocess import person

# model will output multiple(15) results/sec, how are we gonna handle it?

In [67]:
importlib.reload(SLR_model)

<module 'SLR_model' from 'c:\\Users\\jerry\\Desktop\\hly\\2024-2\\cap\\BN_SLR\\BomNae-SLR-1\\SLR_model.py'>

In [42]:
# Configurations
model_dir = "saves"
THRESHOLD = 0.5
# batch_size = 16

In [43]:
# 세이브된 모델 로드
load_path = os.path.join(model_dir, "check_09-01-10.keras")
model = SLR_model.load_model(load_path)

In [ ]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]

person_id = 0
# TODO hardcode path
# a video
for j in os.listdir(os.path.join(preprocess.getoutputdir(),str(person_id))):
    l_raws, r_raws, p_raws, y_raw = preprocess.load_data(f"{person_id}/{j}")
    
    l_raws.append(l_raws)
    r_raws.append(r_raws)
    p_raws.append(p_raws)

In [44]:
# dummy x_raw data
l_raws = []
duration_list = []
h_elem = np.zeros(shape=(4,5,3))
# p_elem = np.zeros(shape=(3,10))
# comb_elem = [h_elem, h_elem, p_elem]
x_elem = []
duration = (np.random.randint(90,100))
duration_list.append(duration)
for j in range(duration): # frame size
    x_elem.append(h_elem)
    # x_elem.append(comb_elem)
l_raws.append(x_elem)
r_raws = []

h_elem = np.zeros(shape=(4,5,3))
# p_elem = np.zeros(shape=(3,10))
# comb_elem = [h_elem, h_elem, p_elem]
x_elem = []
duration = duration_list[0]
for j in range(duration): # frame size
    x_elem.append(h_elem)
    # x_elem.append(comb_elem)
r_raws.append(x_elem)
p_raws = []

# h_elem = np.zeros(shape=(4,5,3))
p_elem = np.zeros(shape=(3,10))
# comb_elem = [h_elem, h_elem, p_elem]
x_elem = []
duration = duration_list[0]
for j in range(duration): # frame size
    # x_elem.append(h_elem)
    x_elem.append(p_elem)
    # x_elem.append(comb_elem)
p_raws.append(x_elem)

In [45]:
l_test, each = SLR_model.serialize(l_raws)
r_test, each = SLR_model.serialize(r_raws)
p_test, each = SLR_model.serialize(p_raws, stride=2)
x_test = (l_test, r_test, p_test)

# dataset = SLR_model.convert_to_dataset_single(x_test, batch_size)


# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)

In [46]:
x_test[2].shape

(5, 32, 3, 10)

In [48]:
predictions = model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [51]:
predictions

array([[0.65784764, 0.00116189, 0.6552309 , 0.66116965, 0.5075511 ,
        0.18879132, 0.71263397, 0.5180616 , 0.5643203 , 0.43498623,
        0.5490358 , 0.38409314],
       [0.65784764, 0.00116189, 0.6552309 , 0.66116965, 0.5075511 ,
        0.18879132, 0.71263397, 0.5180616 , 0.5643203 , 0.43498623,
        0.5490358 , 0.38409314],
       [0.65784764, 0.00116189, 0.6552309 , 0.66116965, 0.5075511 ,
        0.18879132, 0.71263397, 0.5180616 , 0.5643203 , 0.43498623,
        0.5490358 , 0.38409314],
       [0.65784764, 0.00116189, 0.6552309 , 0.66116965, 0.5075511 ,
        0.18879132, 0.71263397, 0.5180616 , 0.5643203 , 0.43498623,
        0.5490358 , 0.38409314],
       [0.65784764, 0.00116189, 0.6552309 , 0.66116965, 0.5075511 ,
        0.18879132, 0.71263397, 0.5180616 , 0.5643203 , 0.43498623,
        0.5490358 , 0.38409314]], dtype=float32)

In [55]:
pred_bin = predictions.round().astype(np.int32)
pred_bin

array([[1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]])

In [76]:
importlib.reload(SLR_model)

<module 'SLR_model' from 'c:\\Users\\jerry\\Desktop\\hly\\2024-2\\cap\\BN_SLR\\BomNae-SLR-1\\SLR_model.py'>

In [78]:
SLR_model.bin2d_to_num(pred_bin.astype(np.int32))

[3002, 3002, 3002, 3002, 3002]

In [79]:
SLR_model.bin2num([1,0,1,1,1,0,1,1,1,0,1,0])

3002

In [ ]:
model.predict